# Classification - Analyze

In [1]:
import yaml
import os
import shutil
import subprocess
import datetime
import numpy
import random
import sklearn.metrics
import tensorflow

print("INFO> TensorFlow version: %s" % tensorflow.__version__)
print("INFO> Num GPUs Available: ", len(tensorflow.config.experimental.list_physical_devices('GPU')))

INFO> TensorFlow version: 1.14.0
INFO> Num GPUs Available:  1


In [2]:
# Read parameters from local config.yaml file, and update corresponding Python variables
currentDir = os.getcwd()
print("INFO> Reading file config.yam from directory: %s" % currentDir)
yamlFile = open('config.yaml','r')
yamlData = yaml.load(yamlFile,Loader=yaml.Loader)

for key in sorted(yamlData):
    print("INFO> %-15s: %s" % (key,yamlData[key]))

batchSize = yamlData['batchSize']
checkDataset = yamlData['checkDataset']
checkpointDir = yamlData['checkpointDir']
createDataset = yamlData['createDataset']
datasetDir = yamlData['datasetDir']
goldenDataset = yamlData['goldenDataset']
imageHeight = yamlData['imageHeight']
imageWidth = yamlData['imageWidth']
learningRate = float(yamlData['learningRate'])
logDir = yamlData['logDir']
nEpochs = yamlData['nEpochs']
nTrnSamples = yamlData['nTrnSamples']
nTstSamples = yamlData['nTstSamples']
nValSamples = yamlData['nValSamples']
remDir = os.path.join(yamlData['datasetDir'], yamlData['remDir'])
tmpDir = os.path.join(yamlData['datasetDir'], yamlData['tmpDir'])
trnDir = os.path.join(yamlData['datasetDir'], yamlData['trnDir'])
tstDir = os.path.join(yamlData['datasetDir'], yamlData['tstDir'])
valDir = os.path.join(yamlData['datasetDir'], yamlData['valDir'])

INFO> Reading file config.yam from directory: /raid5/disk1/mlproj10/classification
INFO> batchSize      : 16
INFO> checkDataset   : True
INFO> checkpointDir  : /home/jmv/data/mlproj10/tmp/
INFO> createDataset  : True
INFO> datasetDir     : /home/jmv/data/mlproj10/dataset
INFO> goldenDataset  : /home/jmv/data/mlproj10/LeryPosesGolden/dataset
INFO> imageHeight    : 360
INFO> imageWidth     : 640
INFO> learningRate   : 1e-6
INFO> logDir         : /home/jmv/data/mlproj10/log/
INFO> nEpochs        : 1000
INFO> nTrnSamples    : 2000
INFO> nTstSamples    : 671
INFO> nValSamples    : 671
INFO> remDir         : rem
INFO> tmpDir         : tmp
INFO> trnDir         : trn
INFO> tstDir         : tst
INFO> valDir         : val


In [ ]:
# Look in the tmp directory and select best model candidate based on train/val loss & accuracy
# For TF1.14, added compile=False. This is not needed for TF2
model = tensorflow.keras.models.load_model('/home/jmv/data/mlproj10/tmp/20200308-150427/00042_0.149766_0.945500_0.211008_0.925305.h5', compile=False)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
tstDataGen = tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

tstGenerator = tstDataGen.flow_from_directory(
    directory=tstDir,
    target_size=(imageHeight,imageWidth),
    batch_size=1,
    class_mode=None,
    shuffle=False)

# Confusion matrix
predictions = model.predict_generator(tstGenerator,numpy.math.ceil(tstGenerator.samples/tstGenerator.batch_size))
images = tstGenerator.filenames
trueClasses = tstGenerator.classes
predictedClasses = numpy.argmax(predictions, axis=1)

report = sklearn.metrics.confusion_matrix(trueClasses, predictedClasses)

print(tstGenerator.class_indices)
print(report)

# List images which have a different predicted class vs. true class
for image, trueClass, predictedClass in zip(images,trueClasses,predictedClasses):
    if trueClass!=predictedClass:
        print("Image: %s, True Class: %d, Predicted Class: %d" % (image, trueClass, predictedClass))

In [ ]:
remDataGen = tensorflow.keras.preprocessing.image.ImageDataGenerator(rescale=1. / 255)

remGenerator = remDataGen.flow_from_directory(
    directory=remDir,
    target_size=(imageHeight,imageWidth),
    batch_size=1,
    class_mode=None,
    shuffle=False)

# Confusion matrix
predictions = model.predict_generator(remGenerator,numpy.math.ceil(remGenerator.samples/remGenerator.batch_size))
images = remGenerator.filenames
trueClasses = remGenerator.classes
predictedClasses = numpy.argmax(predictions, axis=1)

report = sklearn.metrics.confusion_matrix(trueClasses, predictedClasses)

print(remGenerator.class_indices)
print(report)

for image, trueClass, predictedClass in zip(images,trueClasses,predictedClasses):
    if trueClass!=predictedClass:
        print(\"Image: %s, True Class: %d, Predicted Class: %d\" % (image, trueClass, predictedClass))"

In [ ]:
# From book \"Deep Learning w/ Python\" by François Chollet
# From https://stackoverflow.com/questions/58322147/how-to-generate-cnn-heatmaps-using-built-in-keras-in-tf2-0-tf-keras
import matplotlib.pyplot
import cv2
import PIL

def plot_activation(imagePath):
    # Loads an image into PIL format
    myImage = tensorflow.keras.preprocessing.image.load_img(imagePath,target_size=(imageHeight,imageWidth))
    # Converts the PIL image into a Numpy array
    myImageAsArray = tensorflow.keras.preprocessing.image.img_to_array(myImage)
    # Creates a list containing a single image [myImageAsArray]
    myImageAsArray = numpy.expand_dims(myImageAsArray,axis=0)
    # Scales the image in the same way as what we did before the training
    myImageAsArray /= 255.0
    # Gets the result of the model    
    myPrediction = model.predict(myImageAsArray)    
    myPredictedClass = numpy.argmax(myPrediction, axis=1)    
    #print(f\"DBG> Predicted class: {myPredictedClass[0]}\")    
    #    
    convLayer = model.get_layer(\"block5_conv3\")    
    #print(\"DBG> convLayer is\",convLayer)    
    modelOutput = model.output[:,myPredictedClass[0]]    
    # Was forced to add tensorflow.cast(...,'float32') because otherwise the tensor is missing    
    # dtype set to float32. Bug with TF1.14?    
    #grads = tensorflow.cast(tensorflow.keras.backend.gradients(modelOutput,convLayer.output),'float32')    
    grads = tensorflow.keras.backend.gradients(modelOutput,convLayer.output)[0]    
    pooledGrads = tensorflow.keras.backend.mean(grads,axis=(0,1,2))    
    iterate = tensorflow.keras.backend.function([model.input],[pooledGrads,convLayer.output[0]])    
    pooledGradsValue, convLayerOutputValue = iterate([myImageAsArray])    
    numberOfChannelsConvLayer = convLayer.output[0].get_shape()[2]    
    for i in range(numberOfChannelsConvLayer):    
        convLayerOutputValue[:,:,i] *= pooledGradsValue[i]    
    heatMap = numpy.mean(convLayerOutputValue, axis=-1)    
    heatMap = numpy.maximum(heatMap,0)    
    heatMap /= numpy.max(heatMap)    
    matplotlib.pyplot.matshow(heatMap)    
    #    
    img = cv2.imread(imagePath)    
    heatMap = cv2.resize(heatMap,(img.shape[1],img.shape[0]))    
    heatMap =numpy.uint8(255*heatMap)    
    heatMap = cv2.applyColorMap(heatMap,cv2.COLORMAP_JET)    
    superImposedImg = heatMap*0.4+img    
    cv2.imwrite('/home/jmv/data/mlproj8/myresultingimage.jpg',superImposedImg)"

In [ ]:
desiredNumberOfImagesToDisplay = 10    
dirToDisplayFrom = tstDir    
print(f\"DBG> Desired number of images: {desiredNumberOfImagesToDisplay}\")    
print(f\"DBG> Directory to display images from: {dirToDisplayFrom}\")    
  
for myClass in sorted(os.listdir(dirToDisplayFrom)):    
    print(f\"DBG> Class={myClass}\")    
    listOfImages = [image for image in sorted(os.listdir(os.path.join(tstDir,myClass))) if \"copy\" not in image]    
    actualNumberOfImagesToDisplay = len(listOfImages)    
    print(f\"DBG> Actual number of images: {actualNumberOfImagesToDisplay}\")    
    for image in listOfImages[:min(desiredNumberOfImagesToDisplay,actualNumberOfImagesToDisplay)]:    
        print(f\"DBG> Image: {image}\")    
        selectImage = os.path.join(tstDir,myClass,image)    
        plot_activation(selectImage)    
        pil_img = PIL.Image.open('/home/jmv/data/mlproj8/myresultingimage.jpg')    
        myImShow = matplotlib.pyplot.imshow(pil_img)    
        matplotlib.pyplot.title(selectImage,pad=30)    